## 为什么会报错？
```
RuntimeError: Output 0 of UnbindBackward0 is a view and its base or another view of its base has been modified inplace. This view is the output of a function that returns multiple views. Such functions do not allow the output views to be modified inplace. You should replace the inplace operation by an out-of-place one.
```

In [1]:
import torch
import numpy as np
import phi.torch.flow as ptf

import tqdm

device = torch.device('cuda')

In [2]:
BOUNDS = ptf.Box(x=1, y=1, z=1)
RESOLUTION = 100

In [3]:
x = torch.linspace(0.2, 0.8, 3, dtype=torch.float32, device=device)
y = torch.linspace(0.2, 0.8, 3, dtype=torch.float32, device=device)
z = torch.linspace(0.2, 0.8, 3, dtype=torch.float32, device=device)
grid_x, grid_y, grid_z = torch.meshgrid(x, y, z, indexing='ij')

LOCATIONS = torch.stack([grid_x, grid_y, grid_z], dim=-1).reshape(-1, 3)  # [27, 3]
DIRECTIONS = torch.randn_like(LOCATIONS, dtype=torch.float32, device=device, requires_grad=True)  # [27, 3]
OPTIMIZER = torch.optim.RAdam([DIRECTIONS], lr=0.001)

def my_func(xx):
    ret = A * ptf.math.exp(-ptf.length(xx - loca) ** 2 / (2 * sigma ** 2))
    return ptf.vec(x=ret, y=ret, z=ret)

A = 0.2  # 高斯幅值
sigma = 0.2  # 标准差，控制衰减范围
LAMBDA_FIELDS = []
for loca in LOCATIONS:
    LAMBDA = ptf.CenteredGrid(
        values=my_func,
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    LAMBDA_FIELDS.append(LAMBDA)

DIRECTION_FIELDS = []
for dire in DIRECTIONS:
    DIRECTION = ptf.CenteredGrid(
        values=ptf.vec(x=dire[0], y=dire[1], z=dire[2]),
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    DIRECTION_FIELDS.append(DIRECTION)

In [4]:
print(LAMBDA_FIELDS[0].shape)
print(DIRECTION_FIELDS[0].shape)

(xˢ=100, yˢ=100, zˢ=100, vectorᶜ=x,y,z)
(xˢ=100, yˢ=100, zˢ=100, vectorᶜ=x,y,z)
